# Pengenalan Pemrosesan Bahasa Alami

Oleh: Ali Akbar Septiandri

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

## Memuat Dokumen

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    'alt.atheism',
    'soc.religion.christian',
    'comp.sys.ibm.pc.hardware',
    'comp.windows.x',
    'rec.sport.baseball',
    'rec.sport.hockey',
]
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=categories)

In [ ]:
X_train, y_train = newsgroups_train['data'], newsgroups_train['target']
X_test, y_test = newsgroups_test['data'], newsgroups_test['target']

In [ ]:
newsgroups_train['target_names']

## Tokenizing, Stemming, Lemmatizing

Umumnya, kata "token" merujuk pada satuan kata. Namun, tidak jarang kita perlu memecah paragraf menjadi kalimat sehingga kita memerlukan *sentence tokenizer*. Contoh penggunaannya dengan NLTK adalah sebagai berikut.

In [ ]:
import nltk

nltk.sent_tokenize(X_train[3])

In [ ]:
sentence = nltk.sent_tokenize(X_train[3])[1]

### spaCy Tokenizer

Untuk berbagai bahasa (termasuk bahasa Indonesia), salah satu tokenizer terbaik yang dapat digunakan adalah dari spaCy. Dengan satu kali masukan, Anda dapat menghasilkan token yang langsung mempunyai atribut seperti dijelaskan di [dokumentasinya](https://spacy.io/api/token#attributes).

In [ ]:
from spacy.lang.en import English

nlp = English()
[token for token in nlp(sentence) if not token.is_space]

Perhatikan bahwa tanda baca juga dihitung sebagai token!

### Lemmatizer

Dengan menggunakan kode yang sama, Anda hanya perlu mengganti bagian `print(token)` menjadi `print(token.lemma_)` untuk melihat bentuk kamus dari tiap token.

In [ ]:
[token.lemma_ for token in nlp(sentence) if not token.is_space]

### Stemmer

Stemmer berfungsi untuk memotong imbuhan. Di pengolahan teks modern, prapemrosesan ini jarang dilakukan.

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()
tokens = ['regarding', 'programming', 'denied', 'flew']
[stemmer.stem(token) for token in tokens]

## POS Tagging

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')

rows = []
for token in doc:
    rows.append([token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop])
pd.DataFrame(rows, columns=['text','lemma','pos','tag','dep','shape','alpha','stopword'])

## Klasifikasi Dokumen

Pertama, kita sebaiknya melihat sebaran dari kategori dokumen pada data latih.

In [ ]:
sns.countplot(y=y_train)
plt.yticks(range(6), newsgroups_train['target_names']);

Datanya terlihat tersebar cukup merata, kecuali untuk topik `alt.atheism`. Namun, kita tidak perlu menyeimbangkan kelasnya untuk saat ini dan kita dapat menggunakan akurasi sebagai metrik evaluasi.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()
vec.fit_transform(X_train)

Hasil transformasi bag-of-words dari teks menghasilkan *sparse matrix*. Informasi di tiap sel dalam matriks tersebut adalah jumlah kemunculan suatu kata dalam suatu dokumen. Sekarang, kita akan mencoba melakukan klasifikasi datanya.

In [ ]:
clf = make_pipeline(
    CountVectorizer(),
    LogisticRegression(solver='lbfgs', max_iter=300, multi_class='auto', random_state=42)
)
acc = cross_val_score(clf, X_train, y_train, cv=3, n_jobs=2)
print('Akurasi: {:.2%} ± {:.2%}'.format(acc.mean(), acc.std()))

Apa kelemahan dari model ini? Beberapa hal yang dapat dicoba:

1. Bag-of-words $\rightarrow$ n-gram
2. Bag-of-words $\rightarrow$ TF-IDF
3. Buang stopwords
4. Tokenizer $\rightarrow$ Lemmatizer

In [ ]:
nlp = English()

def lemmatizer(text):
    """Mengembalikan list of lemmas"""
    pass

In [ ]:
# Kode Anda di sini

## Reduksi Dimensi

Salah satu cara mudah untuk mengevaluasi data sebelum melakukan pemodelan adalah dengan visualisasi. Namun, dengan dimensi yang begitu besar, bagaimana cara memvisualisasikannya sementara manusia hanya baik mengolah gambar dalam dua dimensi?

Untuk kebutuhan tersebut, Anda dapat menggunakan metode reduksi dimensi seperti *Principal Component Analysis* (PCA). Dalam dunia pengolahan teks, menerapkan PCA pada matriks hasil n-gram dikenal juga dengan nama *Latent Semantic Analysis* (LSA). Berhubung matriks yang digunakan jarang, alternatif PCA yang digunakan adalah [Truncated SVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html).

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE

pipe = make_pipeline(
    TfidfVectorizer(stop_words='english'),
    TruncatedSVD(2, random_state=42)
)
X_map = pipe.fit_transform(X_train)

In [ ]:
X_map.shape

Sekarang, kita sudah dapat memvisualisasikan datanya.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
for i, label in enumerate(newsgroups_train['target_names']):
    ax.scatter(*X_map[y_train == i].T, marker='.', label=label)
plt.legend();

Perhatikan bahwa dokumen-dokumen `rec.sport` berdekatan, begitu juga dokumen-dokumen di bawah kategori `comp`. Sudah sesuai dengan intuisi kita bukan? Pertanyaannya, apakah modelnya juga akan lebih baik dengan LSA?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = make_pipeline(
    TfidfVectorizer(stop_words='english'),
    TruncatedSVD(100, random_state=42),
    LogisticRegression(solver='lbfgs', max_iter=300, multi_class='auto', random_state=42)
)
acc = cross_val_score(clf, X_train, y_train, cv=3, n_jobs=2)
print('Akurasi: {:.2%} ± {:.2%}'.format(acc.mean(), acc.std()))

## Word2Vec

In [ ]:
sentences = []
for doc in X_train:
    sentences.append([token for token in nlp(doc) if not token.is_space])

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy

nlp = spacy.load('en_core_web_md')

In [ ]:
for token in nlp('cat dog banana'):
    print(token.vector.shape)